In [2]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import randForest as rfr
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [3]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [29]:
DS = pd.read_csv('datasetBasin1.csv', index_col = None)
print(DS.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2493980 entries, 0 to 2493979
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
 7   x_coord     int64  
 8   y_coord     int64  
dtypes: float64(6), int64(3)
memory usage: 171.2 MB
None


In [30]:
DS = DS.dropna()
print(DS.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2479538 entries, 5 to 2493974
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
 7   x_coord     int64  
 8   y_coord     int64  
dtypes: float64(6), int64(3)
memory usage: 189.2 MB
None


In [32]:
DS.to_csv('datasetBasin1_NoDataFree.csv', index = None)

In [33]:
DS.describe()

,percentage,disToRiv,TWI,TPI,FAcc,slope,elevation,x_coord,y_coord
count,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06
mean,9.201795e-02,2.084064e+02,9.483515e+00,-3.328848e-04,4.352760e+04,4.821367e+00,1.453463e+02,3.650828e+05,5.264239e+06
std,6.610626e-01,1.713850e+02,4.081659e+00,1.949152e-01,1.031605e+06,4.994698e+00,4.569706e+01,3.169308e+03,1.824893e+03
min,0.000000e+00,0.000000e+00,-9.252120e+00,-7.461320e+00,2.500000e+01,4.700000e-04,-3.685000e-02,3.586780e+05,5.260218e+06
25%,0.000000e+00,7.382411e+01,5.553190e+00,-6.226000e-02,8.892947e+01,1.600760e+00,1.157656e+02,3.623130e+05,5.262723e+06
50%,0.000000e+00,1.677051e+02,1.025636e+01,-2.700000e-04,2.881975e+02,3.192175e+00,1.481632e+02,3.650730e+05,5.264348e+06
75%,0.000000e+00,3.010399e+02,1.289330e+01,6.091000e-02,1.265309e+03,6.192805e+00,1.797259e+02,3.676280e+05,5.265783e+06
max,5.000000e+00,1.349713e+03,2.486353e+01,6.706970e+00,6.282450e+07,7.002943e+01,2.605691e+02,3.721180e+05,5.267978e+06


## balanced sampling

In [34]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

X,Y = ms.importDataSet('datasetBasin1_NoDataFree.csv', 'percentage')

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

TRAIN: 1983630 TEST: [1149772 1110309   98254 ... 2117964  401271  124321]


In [35]:
print( y_train.value_counts())
print(y_test.value_counts())

0    1941324
5      35056
1       7250
Name: percentage, dtype: int64
0    485332
5      8764
1      1812
Name: percentage, dtype: int64


In [36]:
## This proportions are the reason why a sample_weight of 0.01 for the majority class give best results for regression

totalTrain = sum([1941324, 35056, 7250])
totalValidation = sum([485332,8764, 1812])
print("Balance summary of train dataset")
print(f"Class 1% : {1941324/totalTrain}, Class 1%: {7250/totalTrain}, Class 5%: {35056/totalTrain}")
print(f"Class 1% : {485332/totalValidation}, Class 1%: {1812/totalValidation}, Class 5%: {8764/totalValidation}")

Balance summary of train dataset
Class 1% : 0.978672433871236, Class 1%: 0.0036549154832302396, Class 5%: 0.017672650645533694
Class 1% : 0.9786734636263178, Class 1%: 0.003653903546625584, Class 5%: 0.017672632827056632


In [45]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler 

sm = RandomUnderSampler(random_state=50,sampling_strategy='majority')
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({5: 35056, 0: 7250, 1: 7250})


In [43]:
X_res['percentage'] = y_res
print(X_res.head())
X_res.to_csv('basin1TrainingBanlanced.csv', index=None)

X,Y = ms.importDataSet('basin1TrainingBanlanced.csv', 'percentage')

    disToRiv       TWI      TPI         FAcc     slope  elevation  x_coord  \
0  395.37958  16.60988 -0.15622  16352.31836   2.55812  128.22772   367968   
1  140.08926   4.47640  0.07680    195.21506   9.22754  108.65261   367903   
2  105.00000   4.83475  0.37084    146.69675  11.37751   65.99345   367583   
3   25.00000  10.97939 -0.00449  27846.66602   0.28085  186.80829   360483   
4   90.13878   5.02442 -0.19621    121.64829   9.13241  124.75165   364448   

   y_coord  percentage  
0  5263198           0  
1  5263573           0  
2  5265203           0  
3  5263168           0  
4  5264078           0  


In [47]:
print('Resampled dataset shape %s' % Counter(Y))

Resampled dataset shape Counter({0: 7250, 1: 7250, 5: 7250})


In [ ]:
x_validation.to_csv('basin1Validation.csv', index = None)

In [ ]:
x_train['percentage'] = y_train
print(x_train.head())

In [ ]:
x_train.to_csv('basin1Train.csv', index = None)

In [ ]:
RFClassifier = rfr.implementRandomForestCalssifier()

In [ ]:
model = rfr.loadModel('./bestModels/21-46-40 Classifier/2207232146.pkl')

In [ ]:
print(model)

# Data description and visualization

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:
##3 Al features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',
targets = DS['percentage']
# targets = np.where( targets == 5, 2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc_norm']
S = DS['disToRiv']
TWI = DS['TWI']
TPI = DS['TPI']

fig, axs = plt.subplots(3, 2, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[0, 1].scatter(S,targets)
axs[0, 1].set_title("Distance to river")
axs[1, 1].scatter(TWI,targets)
axs[1, 1].set_title("TWI")
axs[2, 1].scatter(TPI,targets)
axs[2, 1].set_title('TPI')
fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] print(dsArray.head())
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
# classifier = rfr.loadModel('./outputs/2022-08-02/11-01-47/2208021101.pkl')
x_test,y_test = ms.importDataSet('basin1Validation.csv', 'percentage')
y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
rfr.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)